In [1]:
import argparse
import numpy as np
import tensorflow as tf

from scada_forecast.preprocess import read_humidity, read_temperature, read_scada
from scada_forecast.preprocess import merge_dataframes, add_calendar_features, prepare_data

from scada_forecast.model import create_model, train_model, inference

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
def hourahead(task, 
              scada_path='data/scada/Dữ liệu SCADA Phụ tải 26.08.2020.xlsx', 
              use_humidity=True,
              humidity_path='data/scada/DoAm.xlsx', 
              use_temperature=True, 
              temperature_path='data/NhietDoQuaKhu.xlsx', 
              use_lunar=True, use_holiday=False, 
              n_val=100 * 24 * 12, n_test=100 * 24 * 12, 
              ckpt_path='tmp/ckpt'):
    df_humidity = None
    df_temperature = None
    df_scada = None
    hour_steps = int(60 / 5)
    input_width = hour_steps
    categorical_unique_values_dict = {
        'Month': list(range(1, 12 + 1)),
        'DayOfYear': list(range(1, 365 + 1)),
        'DayOfWeek': list(range(7)),
        'Hour': list(range(24)),
        'LunarMonth': list(range(1, 12 + 1)),
        'LunarDayOfMonth': list(range(1, 30 + 1)),
        'HolidayCat': list(range(12))
    }
    
    # TIEN XU LY
    print('Doc du lieu scada...')
    df_scada = read_scada(scada_path)
    
    if use_humidity:
        print('Doc du lieu do am...')
        df_humidity = read_humidity(humidity_path)
    if use_temperature:
        print('Doc du lieu nhiet do...')
        df_temperature = read_temperature(temperature_path)
        
    df = merge_dataframes(df_scada, df_temp=df_temperature, df_humidity=df_humidity)
    print('Them cac feature thoi gian (gio, thu, ngay, thang)...')
    df = add_calendar_features(df, use_lunar=use_lunar, use_holiday=use_holiday)
    
    print('Chuan bi du lieu cho mo hinh...')
    df, features_dict, categorical_cols, numeric_cols, target_col = prepare_data(df, 
                                                                                 hour_steps=hour_steps, 
                                                                                 use_temp=use_temperature, 
                                                                                 use_humidity=use_humidity, 
                                                                                 use_lunar=use_lunar,
                                                                                 use_holiday=use_holiday)
    
    dtype_dict = {col: df[col].dtype for col in categorical_cols + numeric_cols}
    chosen_features = (features_dict['calendar_features'] + features_dict['lag_features'] + 
                       features_dict['temperature_features'] + features_dict['humidity_features'])
    
    # TRAIN MO HINH
    if task == 'train':
        print('Training...')
        train_model(df, input_width, n_val, n_test, hour_steps,
                    dtype_dict, categorical_unique_values_dict, 
                    chosen_features, categorical_cols, 
                    numeric_cols, target_col, ckpt_path)
        print('Hoan thanh.')
        
    # DU BAO
    if task == 'inference':
        print('Inference...')
        forecast_df = inference(df, input_width, n_test, hour_steps, ckpt_path,
                                dtype_dict, categorical_unique_values_dict, 
                                chosen_features, categorical_cols, numeric_cols, target_col)
        print(forecast_df)
        return forecast_df

In [3]:
forecast_df = hourahead(task='inference', 
                        scada_path='data/scada/Dữ liệu SCADA Phụ tải 26.08.2020.xlsx', 
                        use_humidity=True,
                        humidity_path='data/scada/DoAm.xlsx', 
                        use_temperature=True, 
                        temperature_path='data/NhietDoQuaKhu.xlsx', 
                        use_lunar=True, use_holiday=False, 
                        n_val=100 * 24 * 12, n_test=100 * 24 * 12, 
                        ckpt_path='tmp/ckpt')

Doc du lieu scada...
Doc du lieu do am...
Doc du lieu nhiet do...
Them cac feature thoi gian (gio, thu, ngay, thang)...


F:\Anaconda3\envs\new_sklearn\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel



Chuan bi du lieu cho mo hinh...
Inference...


F:\Anaconda3\envs\new_sklearn\lib\site-packages\tensorflow\python\keras\engine\functional.py:543: UserWarning: Input dict contained keys ['LogLoad'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


                         Forecast
2020-02-22 00:05:00  23492.824219
2020-02-22 00:10:00  23607.882812
2020-02-22 00:15:00  23294.312500
2020-02-22 00:20:00  23360.541016
2020-02-22 00:25:00  23312.646484
...                           ...
2020-05-31 23:40:00  27212.208984
2020-05-31 23:45:00  26990.041016
2020-05-31 23:50:00  26818.083984
2020-05-31 23:55:00  26602.177734
2020-06-01 00:00:00  26518.613281

[28800 rows x 1 columns]


In [4]:
forecast_df

,Forecast
2020-02-22 00:05:00,23492.824219
2020-02-22 00:10:00,23607.882812
2020-02-22 00:15:00,23294.312500
2020-02-22 00:20:00,23360.541016
2020-02-22 00:25:00,23312.646484
...,...
2020-05-31 23:40:00,27212.208984
2020-05-31 23:45:00,26990.041016
2020-05-31 23:50:00,26818.083984
2020-05-31 23:55:00,26602.177734
